In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable 
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [113]:
def batch_data(size, dataset):
    in_data, out_data = zip(*dataset)
    in_data_vec = []
    for i in list(in_data):
        vec = []
        for j in range(10):
            vec.append(int(i[j]))
        in_data_vec.append(vec)
    out_data = list(out_data)
    in_arr = np.array(in_data_vec)
    out_arr = np.array(out_data)
    
    in_arr = in_arr.reshape(-1, size, 10)
    in_arr = in_arr.astype(np.float32)

    out_arr = out_arr.reshape(-1, size, 1)
    out_arr = out_arr.astype(np.float32)
    print(in_arr.shape, out_arr.shape)
    batched_data = []
    for i in range(in_arr.shape[0]):
        batched_data.append((torch.from_numpy(in_arr[i]), torch.from_numpy(out_arr[i])))
    return batched_data

In [114]:
LOG_INTERVAL = 10 
dataset = [("{:010b}".format(i), i) for i in range(1024)]
random.shuffle(dataset)
train_set = dataset[:1000]
test_set = dataset[1000:]

train_loader = batch_data(20, train_set)
test_loader = batch_data(24, test_set)

def train(model, optimizer, loss_func, epoch, training_history):
    model.train()                                            #Set training mode 
    for batch, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)      #Make data and correct answers variables in the Network
        optimizer.zero_grad()                                #zero the gradients
        output = model(data)                                 #classify the data
        loss = loss_func(output, target)                     #calculate the loss
        loss.backward()                                      #propogate the weight updates through the network
        optimizer.step()
        if batch % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch * len(data), len(train_loader), 100. * batch / len(train_loader), loss.data[0]))
            training_history.append(((len(train_loader) * epoch) + batch * len(data), loss.data[0]))


def test(model, loss_func, epoch, test_loss_history, test_accuracy_history):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target) #Make data and correct answers variables in the Network
        output = model(data)                                           #classify the data
        test_loss += loss_func(output, target).data[0]                 #calculate loss
        pred = output.data.max(1)[1]                                   #get predictions for the batch using argmax
        #correct += pred.eq(target.data).cpu().sum()                    #total correct anwsers 
    test_loss /= len(test_loader)
    print('\nTest set: Average loss: {:.4f}\n'.format(test_loss))
    test_loss_history.append((epoch, test_loss))
    test_accuracy_history.append((epoch, 100. * correct / len(test_loader)))
    
def learn(model, optimizer, loss_func):
    training_loss = []
    test_loss = []
    test_accuracy = []
    for e in range(EPOCHS):
        train(model, optimizer, loss_func, e + 1, training_loss)
        test(model, loss_func, e + 1, test_loss, test_accuracy)
    return (training_loss, test_loss, test_accuracy)



def visualize_learning(training_loss, test_loss, test_accuracy):
    f1 = plt.figure()
    f2 = plt.figure()
    f3 = plt.figure()
    ax1 = f1.add_subplot(111)
    ax2 = f2.add_subplot(111)
    ax3 = f3.add_subplot(111)

    training_loss_batch, training_loss_values = zip(*training_loss)
    ax1.plot(training_loss_batch, training_loss_values)
    ax1.set_title('Training Loss')
    ax1.set_xlabel("Batch")
    ax1.set_ylabel("Loss")


    test_loss_epoch, test_loss_values = zip(*test_loss)
    ax2.plot(test_loss_epoch, test_loss_values)
    ax2.set_title("Testing Loss")
    ax2.set_xlabel("Batch")
    ax2.set_ylabel("Loss")

    test_accuracy_epoch, test_accuracy_values = zip(*test_accuracy)
    ax3.plot(test_accuracy_epoch, test_accuracy_values)
    ax3.set_title("Testing Accuracy")
    ax3.set_xlabel("Batch")
    ax3.set_ylabel("Accuracy (%)")

    plt.show()

def classify(model, img):
    img = Variable(img, volatile=True)
    output = model(img)
    return output.data.max(1)[1]

def classify_an_example(model):
    img = next(iter(test_loader))[0]
    img_np = img.numpy()[0]
    plt.imshow(img_np.reshape(28,28))
    print()
    print("The image is probably: {}".format(classify(model, img)[0]))
    print()

(50, 20, 10) (50, 20, 1)
(1, 24, 10) (1, 24, 1)


In [147]:
### train MLP
class NMLP(nn.Module):
    '''
    3 Hidden Layers, Narrow Model Width for the Hidden Layers (total hidden neurons 2000)
    '''
    def __init__(self):
        super(NMLP, self).__init__()
        self.fc1 = nn.Linear(10, 25)
        self.fc2 = nn.Linear(25, 20)
        self.fc3 = nn.Linear(20, 15)
        self.fc4 = nn.Linear(15, 10)
        self.fc5 = nn.Linear(10,1)
    def forward(self, x):
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        return F.log_softmax(x, dim=1)

In [148]:
EPOCHS = 150                #@param {type:"integer"}           #Number of times to go through the data set
SGD_MOMENTUM = 0.5        #@param {type:"number"}            #How much it takes to change the direction of the gradient
LEARNING_RATE = 0.001     #@param {type:"number"}            #How far each update pushes the weights
narrow_mlp_model = NMLP()
mlp_optimizer = optim.SGD(narrow_mlp_model.parameters(), lr=LEARNING_RATE, momentum=SGD_MOMENTUM)
print(narrow_mlp_model)
NMLP_TRAINING_LOSS, NMLP_TEST_LOSS, NMLP_TEST_ACCURACY = learn(narrow_mlp_model, mlp_optimizer, nn.MSELoss())

NMLP(
  (fc1): Linear(in_features=10, out_features=25, bias=True)
  (fc2): Linear(in_features=25, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=15, bias=True)
  (fc4): Linear(in_features=15, out_features=10, bias=True)
  (fc5): Linear(in_features=10, out_features=1, bias=True)
)
Train Epoch: 1 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 1 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 1 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 1 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 1 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 2 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 2 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 2 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 2 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 2 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 3 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 3 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 3 [400/50 (40

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:29: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:31: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 7 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 7 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 8 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 8 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 8 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 8 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 8 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 9 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 9 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 9 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 9 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 9 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 10 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 10 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 10 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 10 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 10 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Tr

Train Epoch: 38 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 38 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 38 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 39 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 39 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 39 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 39 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 39 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 40 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 40 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 40 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 40 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 40 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 41 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 41 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 41 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 41 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 41 [800/50 (80%

Train Epoch: 66 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 67 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 67 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 67 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 67 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 67 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 68 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 68 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 68 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 68 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 68 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 69 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 69 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 69 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 69 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 69 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 70 [0/50 (0%)]	Loss: 398070.

Train Epoch: 95 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 96 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 96 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 96 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 96 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 96 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 97 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 97 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 97 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 97 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 97 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 98 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 98 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 98 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 98 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 98 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 99 [0/50 (0%)]	Loss: 398070.

Train Epoch: 125 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 125 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 125 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 126 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 126 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 126 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 126 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 126 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 127 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 127 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 127 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 127 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 127 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 128 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 128 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 128 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 128 [600/50 (60%)]	Loss: 401366.687500
Train Epoch

In [149]:
import convert_pyt_to_adj_list as converter

In [150]:
converter.convert_sd_to_adj("narrow_binary_converter", narrow_mlp_model.state_dict())

In [151]:
### train MLP
class WMLP(nn.Module):
    '''
    3 Hidden Layers, Narrow Model Width for the Hidden Layers (total hidden neurons 2000)
    '''
    def __init__(self):
        super(WMLP, self).__init__()
        self.fc1 = nn.Linear(10, 10)
        self.fc2 = nn.Linear(10, 15)
        self.fc3 = nn.Linear(15, 20)
        self.fc4 = nn.Linear(20, 25)
        self.fc5 = nn.Linear(25,1)
    def forward(self, x):
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        return F.log_softmax(x, dim=1)

In [152]:
EPOCHS = 150              #@param {type:"integer"}           #Number of times to go through the data set
SGD_MOMENTUM = 0.5        #@param {type:"number"}            #How much it takes to change the direction of the gradient
LEARNING_RATE = 0.001     #@param {type:"number"}            #How far each update pushes the weights
wide_mlp_model = WMLP()
mlp_optimizer = optim.SGD(wide_mlp_model.parameters(), lr=LEARNING_RATE, momentum=SGD_MOMENTUM)
print(wide_mlp_model)
NMLP_TRAINING_LOSS, NMLP_TEST_LOSS, NMLP_TEST_ACCURACY = learn(wide_mlp_model, mlp_optimizer, nn.MSELoss())

WMLP(
  (fc1): Linear(in_features=10, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=15, bias=True)
  (fc3): Linear(in_features=15, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=25, bias=True)
  (fc5): Linear(in_features=25, out_features=1, bias=True)
)
Train Epoch: 1 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 1 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 1 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 1 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 1 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 2 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 2 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 2 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 2 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 2 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 3 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 3 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 3 [400/50 (40

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:29: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:31: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 6 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 6 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 6 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 6 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 7 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 7 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 7 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 7 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 7 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 8 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 8 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 8 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 8 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 8 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 9 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 9 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 9 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 9 [600/50 (60%)]	Loss: 401366.68


Test set: Average loss: 332177.1562

Train Epoch: 35 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 35 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 35 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 35 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 35 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 36 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 36 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 36 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 36 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 36 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 37 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 37 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 37 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 37 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 37 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 38 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 38 [200/50 (20%)]	Loss: 324084.

Train Epoch: 68 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 68 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 68 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 68 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 69 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 69 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 69 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 69 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 69 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 70 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 70 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 70 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 70 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 70 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 71 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 71 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 71 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 71 [600/50 (60%

Train Epoch: 100 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 100 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 101 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 101 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 101 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 101 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 101 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 102 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 102 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 102 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 102 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 102 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 103 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 103 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 103 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 103 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 103 [800/50 (80%)]	Loss: 550711.125000

Test set: 

Train Epoch: 129 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 129 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 129 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 130 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 130 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 130 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 130 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 130 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 131 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 131 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 131 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 131 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 131 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 132 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 132 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 132 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 132 [600/50 (60%)]	Loss: 401366.687500
Train Epoch

In [153]:
converter.convert_sd_to_adj("wide_binary_converter", wide_mlp_model.state_dict())

In [154]:
### train MLP
class CMLP(nn.Module):
    '''
    3 Hidden Layers, Narrow Model Width for the Hidden Layers (total hidden neurons 2000)
    '''
    def __init__(self):
        super(CMLP, self).__init__()
        self.fc1 = nn.Linear(10, 18)
        self.fc2 = nn.Linear(18, 18)
        self.fc3 = nn.Linear(18, 18)
        self.fc4 = nn.Linear(18, 18)
        self.fc5 = nn.Linear(18,1)
    def forward(self, x):
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        return F.log_softmax(x, dim=1)

In [155]:
EPOCHS = 150                #@param {type:"integer"}           #Number of times to go through the data set
SGD_MOMENTUM = 0.5        #@param {type:"number"}            #How much it takes to change the direction of the gradient
LEARNING_RATE = 0.001     #@param {type:"number"}            #How far each update pushes the weights
constant_mlp_model = CMLP()
mlp_optimizer = optim.SGD(constant_mlp_model.parameters(), lr=LEARNING_RATE, momentum=SGD_MOMENTUM)
print(constant_mlp_model)
NMLP_TRAINING_LOSS, NMLP_TEST_LOSS, NMLP_TEST_ACCURACY = learn(constant_mlp_model, mlp_optimizer, nn.MSELoss())

CMLP(
  (fc1): Linear(in_features=10, out_features=18, bias=True)
  (fc2): Linear(in_features=18, out_features=18, bias=True)
  (fc3): Linear(in_features=18, out_features=18, bias=True)
  (fc4): Linear(in_features=18, out_features=18, bias=True)
  (fc5): Linear(in_features=18, out_features=1, bias=True)
)
Train Epoch: 1 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 1 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 1 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 1 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 1 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 2 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 2 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 2 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 2 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 2 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 3 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 3 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 3 [400/50 (40

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:29: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:31: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



Test set: Average loss: 332177.1562

Train Epoch: 5 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 5 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 5 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 5 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 5 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 6 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 6 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 6 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 6 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 6 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 7 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 7 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 7 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 7 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 7 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 8 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 8 [200/50 (20%)]	Loss: 324084.062500
Train Epoc

Train Epoch: 33 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 33 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 33 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 34 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 34 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 34 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 34 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 34 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 35 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 35 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 35 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 35 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 35 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 36 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 36 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 36 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 36 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 36 [800/50 (80%

Train Epoch: 61 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 62 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 62 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 62 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 62 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 62 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 63 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 63 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 63 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 63 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 63 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 64 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 64 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 64 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 64 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 64 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 65 [0/50 (0%)]	Loss: 398070.

Train Epoch: 90 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 90 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 91 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 91 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 91 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 91 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 91 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 92 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 92 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 92 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 92 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 92 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 93 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 93 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 93 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 93 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 93 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332

Train Epoch: 119 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 119 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 119 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 119 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 120 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 120 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 120 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 120 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 120 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 121 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 121 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 121 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 121 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 121 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 122 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 122 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 122 [400/50 (40%)]	Loss: 400089.937500
Train Epoch

Train Epoch: 147 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 147 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 147 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 147 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 148 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 148 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 148 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 148 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 148 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 149 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 149 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 149 [400/50 (40%)]	Loss: 400089.937500
Train Epoch: 149 [600/50 (60%)]	Loss: 401366.687500
Train Epoch: 149 [800/50 (80%)]	Loss: 550711.125000

Test set: Average loss: 332177.1562

Train Epoch: 150 [0/50 (0%)]	Loss: 398070.656250
Train Epoch: 150 [200/50 (20%)]	Loss: 324084.062500
Train Epoch: 150 [400/50 (40%)]	Loss: 400089.937500
Train Epoch

In [156]:
converter.convert_sd_to_adj("constant_binary_converter", constant_mlp_model.state_dict())